In [ ]:
import pandas as pd
train = pd.read_csv("../input/eng-data/data (2).csv",header = None)
test = pd.read_csv("../input/arabic1/train1.csv",header = None)

In [ ]:
train.head()

In [ ]:
def change(val):
  if(val=="positive"):
    return 2
  elif(val=="negative"):
    return 0
  elif(val=="neutral"):
    return 1
  else:
    return -1

In [ ]:
test[0] = test[0].apply(change)
train[0] = train[0].apply(change)


In [ ]:
test[0].value_counts()

In [ ]:
train.head()

In [ ]:
train = train.sample(frac=1).reset_index(drop=True)


In [ ]:
train.shape

In [ ]:
valid = train.iloc[95000:,:]
train = train.iloc[:95000,:]

In [ ]:
!pip install transformers

In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [ ]:
def build_model(transformer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(3, activation='softmax')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 10
BATCH_SIZE = 64 * strategy.num_replicas_in_sync
MAX_LEN = 192

In [ ]:
# First load the real tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

In [ ]:
x_train = fast_encode(train[1].astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid[1].astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = fast_encode(test[1].astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = tf.keras.utils.to_categorical(
    train[0].values, num_classes=3
)
y_valid = tf.keras.utils.to_categorical(
    valid[0].values, num_classes=3
)

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [ ]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFBertModel
        .from_pretrained('bert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

In [ ]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

In [ ]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

In [ ]:
value = model.predict(test_dataset, verbose=1)
# value = pd.DataFrame(value)
# value.to_csv('submission.csv', index=False)

In [ ]:
y_classes = value.argmax(axis=-1)

In [ ]:
y_classes[3120:3220]

In [ ]:
y = test[0].values

In [ ]:
sum = 0
for i in range(y.shape[0]):
    if(y[i]==y_classes[i]):
        sum+=1

In [ ]:
sum/y.shape[0]